In [1]:
# ============================================
# Modelo Sigmoide (com escolha automática de modelo)
# Notebook "rodável" (sem argparse / sem terminal)
# Atualizado: PATCH para A em Logístico quando vier NaN/ausente
# ============================================

import numpy as np
import pandas as pd
from pathlib import Path
from scipy.optimize import curve_fit, brentq
from pandas.api.types import is_numeric_dtype

# ============================== CONFIG =================================
ARQUIVO_ENTRADA = Path(r"C:\Users\erika\OneDrive\Área de Trabalho\AVICOLA\LOTES.xlsx")
ARQUIVO_SAIDA   = Path(r"C:\Users\erika\OneDrive\Área de Trabalho\AVICOLA\resultado_modelagem_Sigmoide.csv")

ALVO_PESO  = 2800.0     # alvo para idade ótima
DIA_PREVER = 42         # dia para prever o peso

# limites biológicos
IDADE_MIN_BIO = 28.0
IDADE_MAX_BIO = 65.0
ASSINT_MIN = 2000.0
ASSINT_MAX = 6000.0

print("Entrada:", ARQUIVO_ENTRADA)
print("Saída  :", ARQUIVO_SAIDA)

# ========================= FUNÇÕES AUXILIARES ==========================
def ensure_numeric_pt(series):
    """Converte série para numérica, aceitando strings com vírgula decimal."""
    if is_numeric_dtype(series):
        return pd.to_numeric(series, errors="coerce")
    s = series.astype(str).str.strip()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan})
    s = s.str.replace(".", "", regex=False).str.replace(",", ".", regex=False)
    return pd.to_numeric(s, errors="coerce")

def f_richards(t, A, K, B, M, v):
    t = np.asarray(t, dtype=float)
    return A + (K - A) / (1.0 + np.exp(-B * (t - M))) ** (1.0 / v)

def f_gompertz(t, A, B, M):
    t = np.asarray(t, dtype=float)
    return A * np.exp(-np.exp(-B * (t - M)))

def f_logistico(t, K, A, B):
    t = np.asarray(t, dtype=float)
    return K / (1.0 + A * np.exp(-B * t))

def f_vonb(t, K, A, B):
    t = np.asarray(t, dtype=float)
    return K * np.power((1.0 - A * np.exp(-B * t)), 3)

def safe_curve_fit(func, x, y, p0, bounds=None, maxfev=15000):
    try:
        if bounds is None:
            popt, _ = curve_fit(func, x, y, p0=p0, maxfev=maxfev)
        else:
            popt, _ = curve_fit(func, x, y, p0=p0, bounds=bounds, maxfev=maxfev)
        yhat = func(x, *popt)
        resid = y - yhat
        return popt, resid
    except Exception:
        return None, None

def aic_from_residuals(residuals, k):
    if residuals is None:
        return np.inf
    n = residuals.size
    if n == 0:
        return np.inf
    rss = float(np.sum(residuals ** 2))
    if rss <= 0:
        return np.inf
    return n * np.log(rss / n) + 2 * k

def r2_from_residuals(y, residuals):
    ss_res = float(np.sum(residuals ** 2))
    ss_tot = float(np.sum((y - np.mean(y)) ** 2))
    return 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

def invert_time_for_weight(func, params, alvo, t_min=0.0, t_max=200.0):
    """Encontra t tal que func(t,*params)=alvo via brentq com expansão de intervalo."""
    def g(t):
        return func(np.array([t]), *params)[0] - alvo
    cap, step = 400.0, 50.0
    while t_max <= cap and np.sign(g(t_min)) == np.sign(g(t_max)):
        t_max += step
    if np.sign(g(t_min)) == np.sign(g(t_max)):
        return np.nan
    try:
        root = brentq(g, t_min, t_max, maxiter=200)
        return float(root) if root >= 0 else np.nan
    except Exception:
        return np.nan

# ============================== DADOS ==================================
df_raw = pd.read_excel(ARQUIVO_ENTRADA)

rename_map = {
    "CODIGO_DO_PRODUTOR": "produtor",
    "SEXO_DO_LOTE": "sexo",
    "LINHAGEM": "raca",
    "TIPO_AVIARIO": "tipo_aviario",
    "IDADE": "idade_dias",
    "PESO": "peso",
}
df = df_raw.rename(columns=rename_map)

df["idade_dias"] = ensure_numeric_pt(df["idade_dias"])
df["peso"]       = ensure_numeric_pt(df["peso"])
df = df.dropna(subset=["idade_dias", "peso"]).copy()

# strings limpas nas chaves
for c in ["produtor", "sexo", "raca", "tipo_aviario"]:
    df[c] = df[c].astype(str).str.strip()

# ===== núm. de lotes por (produtor, sexo, raca, tipo_aviario)
col_lote = "NOME_DO_LOTE"
if col_lote not in df.columns:
    raise ValueError("Coluna de identificação do lote não encontrada (esperado 'NOME_DO_LOTE').")

lotes_por_grp = (
    df
    .groupby(["produtor", "sexo", "raca", "tipo_aviario"], dropna=False)[col_lote]
    .nunique()
    .reset_index()
    .rename(columns={col_lote: "num_lotes"})
)

# ===== média por idade (curva do produtor)
df_grp = (
    df
    .groupby(["produtor", "sexo", "raca", "tipo_aviario", "idade_dias"], dropna=False, as_index=False)["peso"]
    .mean()
)

df_grp = df_grp.merge(
    lotes_por_grp,
    on=["produtor", "sexo", "raca", "tipo_aviario"],
    how="left"
)

df = df_grp  # base para modelagem por produtor

chaves_grupo = ["produtor", "sexo", "raca", "tipo_aviario"]
chaves_base  = ["produtor", "sexo", "raca", "tipo_aviario"]

# peso REAL observado aos 42 dias (para fallback do Logístico)
peso_real_42_tbl = df[df["idade_dias"] == 42].copy().set_index(chaves_grupo)["peso"]

# ======================= CURVAS BASE (GOMPERTZ) =======================
print("\nAjustando curvas-base (Gompertz)...")
base_models = {}

for chave, gbase in df.groupby(chaves_base, dropna=False):
    gbase = gbase.sort_values("idade_dias")
    x = gbase["idade_dias"].to_numpy(float)
    y = gbase["peso"].to_numpy(float)

    ymin, ymax = float(np.min(y)), float(np.max(y))
    xmin, xmax = float(np.min(x)), float(np.max(x))
    xmed = float(np.median(x))

    A_low  = max(ymax * 0.9, ASSINT_MIN)
    A_high = min(ymax * 3.0, ASSINT_MAX)

    p0 = [min(max(ymax * 1.10, A_low), A_high), 0.03, xmed]
    bounds = ([A_low, 1e-6, xmin - 20.0],
              [A_high, 1.0, xmax + 20.0])

    pg, rg = safe_curve_fit(f_gompertz, x, y, p0=p0, bounds=bounds)
    if pg is None:
        print("Falha curva-base para chave:", chave)
        continue

    A_base, B_base, M_base = pg
    base_models[chave] = (A_base, B_base, M_base)

    print(f"Base {chave} -> A={A_base:.1f} B={B_base:.4f} M={M_base:.1f}")

def chave_base_from_row(row):
    return (row["produtor"], row["sexo"], row["raca"], row["tipo_aviario"])

# ================= AJUSTE POR PRODUTOR + ESCOLHA DE MODELO ===============
linhas = []

for chave_grp, g in df.groupby(chaves_grupo, dropna=False):

    g = g.sort_values("idade_dias")
    x_all = g["idade_dias"].to_numpy(float)
    y_all = g["peso"].to_numpy(float)
    n = len(x_all)

    ymin, ymax = float(np.min(y_all)), float(np.max(y_all))
    xmin, xmax = float(np.min(x_all)), float(np.max(x_all))
    xmed = float(np.median(x_all))

    uma_linha = g.iloc[0]
    chave_b = chave_base_from_row(uma_linha)
    if chave_b not in base_models:
        chave_b = next(iter(base_models.keys()))
    A_base, B_base_base, M_base_base = base_models[chave_b]

    num_lotes = uma_linha.get("num_lotes", np.nan)

    tipo_ajuste = None
    modelo_escolhido = None
    params_escolhidos = (np.nan,) * 5
    R2 = np.nan
    idade_2800 = np.nan
    idade_2800_ajust = np.nan
    flag_idade_2800 = "sem_solucao"
    peso_42 = np.nan
    f_model = None

    # ========= CASO ESPECIAL: produtor com 1 ou 2 lotes -> usa outros produtores (Gompertz)
    if not pd.isna(num_lotes) and int(num_lotes) <= 2:
        prod0, sexo0, raca0, avi0 = uma_linha["produtor"], uma_linha["sexo"], uma_linha["raca"], uma_linha["tipo_aviario"]

        g_pool = df[
            (df["sexo"] == sexo0) &
            (df["raca"] == raca0) &
            (df["tipo_aviario"] == avi0) &
            (df["produtor"] != prod0)
        ].sort_values("idade_dias")

        if len(g_pool) >= 3:
            tipo_ajuste = "gompertz_outros_produtores"
            modelo_escolhido = "Gompertz_outros"

            x_fit = g_pool["idade_dias"].to_numpy(float)
            y_fit = g_pool["peso"].to_numpy(float)

            ymin_f, ymax_f = float(np.min(y_fit)), float(np.max(y_fit))
            xmin_f, xmax_f = float(np.min(x_fit)), float(np.max(x_fit))
            xmed_f = float(np.median(x_fit))

            A_low  = max(ymax_f * 0.9, ASSINT_MIN)
            A_high = min(ymax_f * 3.0, ASSINT_MAX)

            p0_g = [min(max(ymax_f * 1.05, A_low), A_high), 0.03, xmed_f]
            bounds_g = ([A_low, 1e-6, xmin_f - 20.0],
                        [A_high, 1.0, xmax_f + 20.0])

            pg, rg = safe_curve_fit(f_gompertz, x_fit, y_fit, p0=p0_g, bounds=bounds_g)
            if pg is not None:
                A_g, B_g, M_g = pg
                params_escolhidos = (float(A_g), np.nan, float(B_g), float(M_g), np.nan)
                f_model = f_gompertz
                yhat_fit = f_model(x_fit, *pg)
                R2 = r2_from_residuals(y_fit, y_fit - yhat_fit)
                idade_2800 = invert_time_for_weight(f_model, pg, ALVO_PESO, t_min=0.0, t_max=max(200.0, xmax_f + 30.0))
                peso_42 = float(f_model(np.array([DIA_PREVER]), *pg)[0])

    # --------- CASO 1: >= 5 pontos -> ajuste completo -----------
    if modelo_escolhido is None and n >= 5:
        tipo_ajuste = "completo"
        candidatos = []

        A_low  = max(ymax * 0.9, ASSINT_MIN)
        A_high = min(ymax * 3.0, ASSINT_MAX)
        K_low, K_high = A_low, A_high

        # Gompertz
        p0_g = [min(max(ymax * 1.05, A_low), A_high), 0.03, xmed]
        bounds_g = ([A_low, 1e-6, xmin - 20.0], [A_high, 1.0, xmax + 20.0])
        pg, rg = safe_curve_fit(f_gompertz, x_all, y_all, p0=p0_g, bounds=bounds_g)
        if pg is not None:
            candidatos.append(("Gompertz", f_gompertz, pg, aic_from_residuals(rg, 3)))

        # Logístico
        p0_l = [min(max(ymax * 1.05, K_low), K_high), 1.0, 0.03]
        bounds_l = ([K_low, 1e-6, 1e-6], [K_high, 50.0, 1.0])
        pl, rl = safe_curve_fit(f_logistico, x_all, y_all, p0=p0_l, bounds=bounds_l)
        if pl is not None:
            candidatos.append(("Logistico", f_logistico, pl, aic_from_residuals(rl, 3)))

        # Von Bertalanffy
        p0_v = [min(max(ymax * 1.05, K_low), K_high), 0.9, 0.02]
        bounds_v = ([K_low, 0.01, 1e-6], [K_high, 2.0, 1.0])
        pv, rv = safe_curve_fit(f_vonb, x_all, y_all, p0=p0_v, bounds=bounds_v)
        if pv is not None:
            candidatos.append(("VonBertalanffy", f_vonb, pv, aic_from_residuals(rv, 3)))

        # Richards
        p0_r = [ymin, min(max(ymax * 1.05, K_low), K_high), 0.03, xmed, 1.2]
        bounds_r = ([0.0, K_low, 1e-6, xmin - 20.0, 0.2], [ymax * 1.2, K_high, 1.0, xmax + 20.0, 5.0])
        pr, rr = safe_curve_fit(f_richards, x_all, y_all, p0=p0_r, bounds=bounds_r)
        if pr is not None:
            candidatos.append(("Richards", f_richards, pr, aic_from_residuals(rr, 5)))

        if candidatos:
            modelo_escolhido, f_sel, params, _aic = sorted(candidatos, key=lambda z: z[3])[0]
            yhat_all = f_sel(x_all, *params)
            R2 = r2_from_residuals(y_all, y_all - yhat_all)

            idade_2800 = invert_time_for_weight(f_sel, params, ALVO_PESO, t_min=0.0, t_max=max(200.0, xmax + 30.0))
            peso_42 = float(f_sel(np.array([DIA_PREVER]), *params)[0])

            # gravação dos parâmetros (tipos float)  // PATCH: floats garantidos
            if modelo_escolhido == "Gompertz":
                A, B, M = params
                params_escolhidos = (float(A), np.nan, float(B), float(M), np.nan)

            elif modelo_escolhido in ("Logistico", "VonBertalanffy"):
                K, A2, B = params

                # ===== PATCH: reconstrução de A quando inválido (Logístico) =====
                if modelo_escolhido == "Logistico":
                    A_fix = float(A2) if np.isfinite(A2) and A2 > 0 else np.nan
                    if not np.isfinite(A_fix) or A_fix <= 0:
                        # tenta y(42) do grupo
                        try:
                            y_ref = float(peso_real_42_tbl.loc[chave_grp])
                            t_ref = 42.0
                        except Exception:
                            if len(x_all) > 0:
                                idx = int(np.argmin(np.abs(x_all - 42.0)))
                                y_ref = float(y_all[idx])
                                t_ref = float(x_all[idx])
                            else:
                                y_ref, t_ref = np.nan, np.nan
                        if np.isfinite(y_ref) and y_ref > 0 and np.isfinite(K) and np.isfinite(B):
                            A_fix = (K / y_ref - 1.0) * np.exp(B * t_ref)
                    A2 = A_fix

                # salva nas colunas (A, K, B, M, v)
                params_escolhidos = (float(A2) if np.isfinite(A2) else np.nan,
                                     float(K), float(B), np.nan, np.nan)

            elif modelo_escolhido == "Richards":
                A, K, B, M, v = params
                params_escolhidos = (float(A), float(K), float(B), float(M), float(v))

            f_model = f_sel

    # --------- CASO 2: 2 a 4 pontos -> Gompertz com A_base fixo -----
    if modelo_escolhido is None and 2 <= n <= 4:
        tipo_ajuste = "parcial_gompertz_base"
        def gompertz_parcial(t, B, M): return f_gompertz(t, A_base, B, M)

        p0 = [0.03, xmed]
        bounds = ([1e-6, xmin - 20.0], [1.0,  xmax + 20.0])
        params_parc, r_parc = safe_curve_fit(gompertz_parcial, x_all, y_all, p0=p0, bounds=bounds)
        if params_parc is not None:
            B_parc, M_parc = params_parc
            modelo_escolhido = "Gompertz_baseA"
            yhat_all = gompertz_parcial(x_all, B_parc, M_parc)
            R2 = r2_from_residuals(y_all, y_all - yhat_all)
            idade_2800 = invert_time_for_weight(lambda t, b, m: f_gompertz(t, A_base, b, m),
                                                (B_parc, M_parc), ALVO_PESO,
                                                t_min=0.0, t_max=max(200.0, xmax + 30.0))
            peso_42 = float(f_gompertz(np.array([DIA_PREVER]), A_base, B_parc, M_parc)[0])
            params_escolhidos = (float(A_base), np.nan, float(B_parc), float(M_parc), np.nan)
            f_model = gompertz_parcial

    # --------- CASO 3: 0 ou 1 ponto -> usa apenas a curva-base ----------
    if modelo_escolhido is None and n <= 1:
        tipo_ajuste = "base_global"
        modelo_escolhido = "Gompertz_base"
        A_b, B_b, M_b = A_base, B_base_base, M_base_base
        params_escolhidos = (float(A_b), np.nan, float(B_b), float(M_b), np.nan)
        R2 = np.nan
        idade_2800 = invert_time_for_weight(f_gompertz, (A_b, B_b, M_b), ALVO_PESO, t_min=0.0, t_max=200.0)
        peso_42 = float(f_gompertz(np.array([DIA_PREVER]), A_b, B_b, M_b)[0])
        f_model = f_gompertz

    # ===== critério biológico para idade 2800
    if not np.isnan(idade_2800):
        flag_idade_2800 = "ok"
        idade_2800_ajust = idade_2800
        if idade_2800 < IDADE_MIN_BIO:
            flag_idade_2800 = "muito_baixa"; idade_2800_ajust = np.nan
        elif idade_2800 > IDADE_MAX_BIO:
            flag_idade_2800 = "muito_alta";  idade_2800_ajust = np.nan
    else:
        flag_idade_2800 = "sem_solucao"; idade_2800_ajust = np.nan

    # ===== LOOCV (quando aplicável)
    R2_cv = np.nan
    rmse_cv = np.nan
    if (f_model is not None and modelo_escolhido is not None and n >= 3 and
        tipo_ajuste not in ["base_global", "gompertz_outros_produtores"]):
        preds = []
        ok = True
        for i in range(n):
            mask = np.ones(n, dtype=bool); mask[i] = False
            x_tr, y_tr = x_all[mask], y_all[mask]

            ymin_t, ymax_t = float(np.min(y_tr)), float(np.max(y_tr))
            xmin_t, xmax_t = float(np.min(x_tr)), float(np.max(x_tr))
            xmed_t = float(np.median(x_tr))

            if tipo_ajuste == "completo":
                if modelo_escolhido == "Gompertz":
                    A_low  = max(ymax_t * 0.9, ASSINT_MIN)
                    A_high = min(ymax_t * 3.0, ASSINT_MAX)
                    p0 = [min(max(ymax_t * 1.05, A_low), A_high), 0.03, xmed_t]
                    bounds = ([A_low, 1e-6, xmin_t - 20.0], [A_high, 1.0, xmax_t + 20.0])
                    pfit, _ = safe_curve_fit(f_gompertz, x_tr, y_tr, p0=p0, bounds=bounds)
                    func = f_gompertz
                elif modelo_escolhido == "Logistico":
                    K_low = max(ymax_t * 0.9, ASSINT_MIN); K_high = min(ymax_t * 3.0, ASSINT_MAX)
                    p0 = [min(max(ymax_t * 1.05, K_low), K_high), 1.0, 0.03]
                    bounds = ([K_low, 1e-6, 1e-6], [K_high, 50.0, 1.0])
                    pfit, _ = safe_curve_fit(f_logistico, x_tr, y_tr, p0=p0, bounds=bounds)
                    func = f_logistico
                elif modelo_escolhido == "VonBertalanffy":
                    K_low = max(ymax_t * 0.9, ASSINT_MIN); K_high = min(ymax_t * 3.0, ASSINT_MAX)
                    p0 = [min(max(ymax_t * 1.05, K_low), K_high), 0.9, 0.02]
                    bounds = ([K_low, 0.01, 1e-6], [K_high, 2.0, 1.0])
                    pfit, _ = safe_curve_fit(f_vonb, x_tr, y_tr, p0=p0, bounds=bounds)
                    func = f_vonb
                elif modelo_escolhido == "Richards":
                    K_low = max(ymax_t * 0.9, ASSINT_MIN); K_high = min(ymax_t * 3.0, ASSINT_MAX)
                    p0 = [ymin_t, min(max(ymax_t * 1.05, K_low), K_high), 0.03, xmed_t, 1.2]
                    bounds = ([0.0, K_low, 1e-6, xmin_t - 20.0, 0.2],
                              [ymax_t * 1.2, K_high, 1.0, xmax_t + 20.0, 5.0])
                    pfit, _ = safe_curve_fit(f_richards, x_tr, y_tr, p0=p0, bounds=bounds)
                    func = f_richards
                else:
                    pfit, func = None, None

            elif tipo_ajuste == "parcial_gompertz_base":
                def gompertz_parcial_cv(t, B, M): return f_gompertz(t, A_base, B, M)
                p0 = [0.03, xmed_t]
                bounds = ([1e-6, xmin_t - 20.0], [1.0,  xmax_t + 20.0])
                pfit, _ = safe_curve_fit(gompertz_parcial_cv, x_tr, y_tr, p0=p0, bounds=bounds)
                func = gompertz_parcial_cv
            else:
                pfit, func = None, None

            if pfit is None or func is None:
                ok = False; break

            y_pred_i = func(np.array([x_all[i]]), *pfit)[0]
            preds.append(y_pred_i)

        if ok and len(preds) == n:
            y_pred = np.array(preds, dtype=float)
            resid_cv = y_all - y_pred
            rmse_cv = float(np.sqrt(np.mean(resid_cv ** 2)))
            R2_cv = r2_from_residuals(y_all, resid_cv)

    # ===== assíntota do modelo + flags
    assint = np.nan
    if modelo_escolhido in ["Gompertz", "Gompertz_baseA", "Gompertz_base", "Gompertz_outros"]:
        assint = params_escolhidos[0]  # A
    elif modelo_escolhido in ["Logistico", "VonBertalanffy", "Richards"]:
        assint = params_escolhidos[1]  # K

    flag_assint = "sem_modelo"
    if not np.isnan(assint):
        flag_assint = "ok"
        if assint < ymax * 0.95:
            flag_assint = "menor_que_observado"
        if assint < ASSINT_MIN:
            flag_assint = "muito_baixa"
        elif assint > ASSINT_MAX:
            flag_assint = "muito_alta"

    # ===== peso REAL aos 42 (se houver) para referência
    try:
        peso_real_42 = float(peso_real_42_tbl.loc[chave_grp])
    except Exception:
        peso_real_42 = np.nan

    # ===== monta linha final (tudo como float quando puder)
    A_out, K_out, B_out, M_out, v_out = params_escolhidos
    linha = list(chave_grp) + [
        tipo_ajuste,
        modelo_escolhido,
        float(A_out) if np.isfinite(A_out) else np.nan,
        float(K_out) if np.isfinite(K_out) else np.nan,
        float(B_out) if np.isfinite(B_out) else np.nan,
        float(M_out) if np.isfinite(M_out) else np.nan,
        float(v_out) if np.isfinite(v_out) else np.nan,
        float(assint) if np.isfinite(assint) else np.nan,
        flag_assint,
        float(R2) if np.isfinite(R2) else np.nan,
        float(R2_cv) if np.isfinite(R2_cv) else np.nan,
        float(rmse_cv) if np.isfinite(rmse_cv) else np.nan,
        float(idade_2800) if np.isfinite(idade_2800) else np.nan,
        float(idade_2800_ajust) if np.isfinite(idade_2800_ajust) else np.nan,
        flag_idade_2800,
        float(peso_42) if np.isfinite(peso_42) else np.nan,
        float(peso_real_42) if np.isfinite(peso_real_42) else np.nan,
        int(n)
    ]
    linhas.append(linha)

# ============================= RESULTADO FINAL ==========================
cols_saida = chaves_grupo + [
    "tipo_ajuste",
    "modelo_escolhido",
    "A", "K", "B", "M", "v",
    "ASSINTOTE",
    "FLAG_ASSINTOTE",
    "R2",
    "R2_LOOCV",
    "RMSE_LOOCV",
    "IDADE_PARA_2800G",
    "IDADE_PARA_2800G_AJUST",
    "FLAG_IDADE_2800",
    "peso_previsto_42",
    "peso_real_42",
    "num_pontos"
]

df_saida = pd.DataFrame(linhas, columns=cols_saida).sort_values(
    "IDADE_PARA_2800G_AJUST", ascending=True
)

print("\n=== Saída (primeiras linhas) ===")
display(df_saida.head(20)) if "display" in globals() else print(df_saida.head(20))

df_saida.to_csv(ARQUIVO_SAIDA, index=False, encoding="utf-8-sig")
print("\n✅ Arquivo gerado em:", ARQUIVO_SAIDA)


Entrada: C:\Users\erika\OneDrive\Área de Trabalho\AVICOLA\LOTES.xlsx
Saída  : C:\Users\erika\OneDrive\Área de Trabalho\AVICOLA\resultado_modelagem_Sigmoide.csv

Ajustando curvas-base (Gompertz)...
Base ('10028', 'Fêmeas', 'COBB', 'Convencional 3') -> A=4856.0 B=0.0516 M=30.0
Base ('10028', 'Fêmeas', 'ROSS', 'Convencional 3') -> A=5092.8 B=0.0473 M=33.4
Base ('10028', 'Machos', 'COBB', 'Convencional 3') -> A=6000.0 B=0.0493 M=32.3
Base ('10028', 'Machos', 'HUBBARD', 'Convencional 3') -> A=5172.9 B=0.0475 M=32.2
Base ('10028', 'Mistos', 'COBB', 'Convencional 3') -> A=5519.4 B=0.0488 M=32.9
Base ('10028', 'Mistos', 'ROSS', 'Convencional 3') -> A=5125.6 B=0.0502 M=32.0
Base ('1020', 'Fêmeas', 'COBB', 'Convencional 1') -> A=6000.0 B=0.0426 M=35.1
Base ('1020', 'Machos', 'COBB', 'Convencional 1') -> A=4733.3 B=0.0549 M=28.2
Base ('1020', 'Mistos', 'COBB', 'Convencional 1') -> A=6000.0 B=0.0460 M=34.7
Base ('1020', 'Mistos', 'ROSS', 'Convencional 1') -> A=3942.0 B=0.0614 M=26.2
Base ('1022', 

C:\Users\erika\AppData\Local\Temp\ipykernel_13892\2786833300.py:60: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(func, x, y, p0=p0, bounds=bounds, maxfev=maxfev)


Base ('4564', 'Fêmeas', 'COBB', 'Convencional 3') -> A=5041.2 B=0.0478 M=32.0
Base ('4564', 'Fêmeas', 'ROSS', 'Convencional 3') -> A=5914.5 B=0.0427 M=37.2
Base ('4564', 'Mistos', 'COBB', 'Convencional 3') -> A=4785.1 B=0.0531 M=30.2
Base ('4564', 'Mistos', 'ROSS', 'Convencional 3') -> A=4526.5 B=0.0548 M=29.3
Base ('45693', 'Fêmeas', 'COBB', 'Climatizado 2') -> A=6000.0 B=0.0427 M=35.3
Base ('45693', 'Fêmeas', 'COBB', 'Convencional 3') -> A=5263.3 B=0.0460 M=32.9
Base ('45693', 'Fêmeas', 'ROSS', 'Climatizado 2') -> A=5098.7 B=0.0437 M=34.4
Base ('45693', 'Fêmeas', 'ROSS', 'Convencional 3') -> A=6000.0 B=0.0409 M=38.6
Base ('45693', 'Machos', 'COBB', 'Climatizado 2') -> A=6000.0 B=0.0514 M=31.5
Base ('45693', 'Machos', 'COBB', 'Convencional 3') -> A=6000.0 B=0.0467 M=34.0
Base ('45693', 'Machos', 'ROSS', 'Climatizado 2') -> A=5631.7 B=0.0514 M=31.4
Base ('45693', 'Machos', 'ROSS', 'Convencional 3') -> A=6000.0 B=0.0509 M=33.3
Base ('45693', 'Mistos', 'COBB', 'Climatizado 2') -> A=5268.

C:\Users\erika\AppData\Local\Temp\ipykernel_13892\2786833300.py:60: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(func, x, y, p0=p0, bounds=bounds, maxfev=maxfev)



=== Saída (primeiras linhas) ===
     produtor    sexo  raca    tipo_aviario tipo_ajuste modelo_escolhido  \
1461    58294  Machos  ROSS   Climatizado 2    completo        Logistico   
1275    58213  Machos  COBB   Climatizado 2    completo         Richards   
1250    57814  Machos  COBB   Climatizado 2    completo         Richards   
1713    60211  Machos  COBB  Convencional 2    completo         Richards   
1612    58371  Machos  COBB   Climatizado 2    completo         Gompertz   
522     30030  Machos  COBB   Climatizado 2    completo        Logistico   
1387    58253  Machos  COBB  Convencional 1    completo         Richards   
684     47320  Machos  COBB   Climatizado 2    completo         Richards   
1790    63291  Machos  COBB   Climatizado 2    completo         Gompertz   
1169    55911  Machos  COBB   Climatizado 2    completo         Richards   
1955    69859  Machos  COBB  Convencional 2    completo         Richards   
1392    58254  Machos  COBB  Convencional 3    complet